In [1]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 

In [2]:
# Load data
from tensorflow. keras.datasets import fashion_mnist

(x_train_set, y_train_set), (x_test, y_test) = fashion_mnist.load_data()

#Split data
from sklearn.model_selection import train_test_split 

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_set, y_train_set, random_state=1)

# Preprocessing
x_train = x_train / 255.0
x_valid = x_valid / 255.0 
x_test = x_test  / 255.0

In [3]:
#不用執行，只是預先確認分割的資料結果
print(y_train[:10])# 全部
ind_B = (y_train == 4) | (y_train == 5) #篩選出原分類：4涼鞋，5外套  
print(ind_B[:10])
#找出ind_B=True,且y_train=5的轉成 1.0
#若y_train=4則會變成 0.0
y_B = (y_train[ind_B] == 5).astype(np.float32)
print(y_B[:10])
y_A = y_train[~ind_B] #排除 ind_B，即找出ind_B=False
print(y_A[:10])
y_A[y_A > 5] -= 2 #將原分類結果 > 5的，改成-2
print(y_A[:10])

[0 9 8 2 2 5 1 4 1 1]
[False False False False False  True False  True False False]
[1. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
[0 9 8 2 2 1 1 1 8 9]
[0 7 6 2 2 1 1 1 6 7]


In [4]:
#將 data拆成 2部分
def split_data(x, y) :
    ind_B = (y == 4) | (y == 5) #篩選原分類：4涼鞋，5外套 
    y_B = (y[ind_B] == 5).astype(np.float32)#原分類 5 轉成 1；分類 4轉成 0
    y_A = y[~ind_B] #排除 ind_B
    y_A[y_A > 5] -= 2 #原分類編號>5的，-2
    return (x[~ind_B], y_A), (x[ind_B], y_B)

(x_train_A, y_train_A), (x_train_B, y_train_B) = split_data(x_train, y_train) 
(x_valid_A, y_valid_A), (x_valid_B, y_valid_B) = split_data(x_valid, y_valid)
(x_test_A, y_test_A), (x_test_B, y_test_B) = split_data(x_test, y_test)

x_train_B, y_train_B = x_train_B[:100], y_train_B[:100]

In [5]:
x_train_A.shape, y_train_A[: 10]

((35968, 28, 28), array([0, 7, 6, 2, 2, 1, 1, 1, 6, 7], dtype=uint8))

In [6]:
x_train_B.shape, y_train_B[:10]

((100, 28, 28), array([1., 0., 0., 0., 1., 1., 1., 0., 0., 0.], dtype=float32))

In [7]:
keras.backend.clear_session()
np. random. seed (1)
tf.random.set_seed(1)

### Model A

In [8]:
model_A = keras.models.Sequential ()
#layer 0
model_A.add(keras.layers.Flatten(input_shape= [28, 28])) 
#layer 1-4
for hidden_i in (200, 150, 100, 50):
    model_A.add(keras.layers.Dense(hidden_i, activation='relu'))
#layer5:輸出 8 類
model_A.add(keras.layers.Dense(8, activation='softmax' ))

c:\Users\ELSA\anaconda3\envs\PyIII_12\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model_A.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           408 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,708 (811.36 KB)

 Trainable params: 207,708 (811.36 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model_A. compile (loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.SGD (learning_rate=1e-3), 
                  metrics=['accuracy'])

In [11]:
train = model_A. fit(x_train_A, y_train_A, epochs=20,
                    validation_data=(x_valid_A, y_valid_A))

Epoch 1/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3987 - loss: 1.8035 - val_accuracy: 0.7568 - val_loss: 1.0048
Epoch 2/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7828 - loss: 0.8627 - val_accuracy: 0.7945 - val_loss: 0.6205
Epoch 3/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8020 - loss: 0.5950 - val_accuracy: 0.8074 - val_loss: 0.5431
Epoch 4/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8127 - loss: 0.5310 - val_accuracy: 0.8172 - val_loss: 0.5077
Epoch 5/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8208 - loss: 0.4993 - val_accuracy: 0.8262 - val_loss: 0.4840
Epoch 6/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8289 - loss: 0.4774 - val_accuracy: 0.8350 - val_loss: 0.4656
Epoch 7/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8358 - loss: 0.4601 - val_accuracy: 0.8403 - val_loss: 0.4507
Epoch 8/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8428 - loss: 0.4458 - 

In [12]:
model_A.evaluate(x_test_A, y_test_A)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8714 - loss: 0.3715


[0.3839355409145355, 0.8646249771118164]

In [13]:
model_A.save ('model_A.h5')

### Model B

In [14]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape= [28, 28])) 

for hidden_i in (300, 150, 100, 80):
    model_B.add(keras.layers.Dense(hidden_i, activation='relu')) 
    
model_B.add(keras.layers.Dense(1, activation='sigmoid'))

In [15]:
model_B.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 150)            │        45,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 80)             │         8,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 303,911 (1.16 MB)

 Trainable params: 303,911 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model_B.compile(loss='binary_crossentropy',
                optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
                metrics=['accuracy'])

In [17]:
train = model_B.fit(x_train_B, y_train_B, epochs=20,
                    validation_data=(x_valid_B, y_valid_B))

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.5075 - loss: 0.6309 - val_accuracy: 0.4933 - val_loss: 0.6407
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5075 - loss: 0.6283 - val_accuracy: 0.4933 - val_loss: 0.6383
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5075 - loss: 0.6258 - val_accuracy: 0.4933 - val_loss: 0.6359
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5075 - loss: 0.6233 - val_accuracy: 0.4933 - val_loss: 0.6336
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5075 - loss: 0.6209 - val_accuracy: 0.4933 - val_loss: 0.6312
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5075 - loss: 0.6185 - val_accuracy: 0.4933 - val_loss: 0.6289
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5075 - loss: 0.6161 - val_accuracy: 0.4933 - val_loss: 0.6266
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5075 - loss: 0.6137 - val_accuracy: 0.4933 - val_loss: 0.6243


In [18]:
model_B.evaluate(x_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - accuracy: 0.5218 - loss: 0.5816


[0.5923401117324829, 0.5009999871253967]

### 載入模型

In [19]:
# Model_tr (Transfer Learning) 
#1. Load model_A 載入模型A
model_A = keras.models.load_model('model_A.h5')

In [20]:
model_A.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           408 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,710 (811.37 KB)

 Trainable params: 207,708 (811.36 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [21]:
# 2. Reuse Lower Layers 使用較低層
# flatten (Flatten) / dense (Dense) / dense_1 (Dense) / dense_2 (Dense)
model_tr = keras.models.Sequential(model_A.layers[:-2])#不含後二層


In [22]:
model_tr.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,250 (790.04 KB)

 Trainable params: 202,250 (790.04 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# 3. Replace Upper Layers
model_tr.add(keras.layers.Dense(80, activation='relu')) 
model_tr.add(keras.layers.Dense(50, activation='relu')) 
model_tr.add(keras.layers.Dense(1, activation='sigmoid'))

In [24]:
model_tr.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,250 (790.04 KB)

 Trainable params: 202,250 (790.04 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# 4. Train in the First Few Epochs 初期訓練
for layer in model_tr.layers[:-3]:
    layer.trainable = False #鎖定不做訓練

In [26]:
model_tr.summary(show_trainable=True)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ flatten (Flatten)           │ (None, 784)           │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 200)           │    157,000 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 150)           │     30,150 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 100)           │     15,100 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_10 (Dense)            │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_11 (Dense)            │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_12 (Dense)            │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 202,250 (790.04 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 202,250 (790.04 KB)

In [27]:
model_tr.compile(loss='binary_crossentropy',
                 optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
                 metrics= ['accuracy'])

In [29]:
train = model_tr.fit(x_train_B, y_train_B, epochs=10,
                      validation_data=(x_valid_B, y_valid_B))

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7910 - loss: 0.5920 - val_accuracy: 0.8386 - val_loss: 0.5576
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8464 - loss: 0.5583 - val_accuracy: 0.8669 - val_loss: 0.5312
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8925 - loss: 0.5305 - val_accuracy: 0.8908 - val_loss: 0.5086
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9201 - loss: 0.5066 - val_accuracy: 0.9077 - val_loss: 0.4889
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9354 - loss: 0.4860 - val_accuracy: 0.9249 - val_loss: 0.4717
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9354 - loss: 0.4680 - val_accuracy: 0.9431 - val_loss: 0.4562
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9354 - loss: 0.4518 - val_accuracy: 0.9555 - val_loss: 0.4421
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9569 - loss: 0.4372 - val_accuracy: 0.9663 - val_loss: 0.4293


In [30]:
# 5. Continue Training后期訓練(全部參數都要參與訓練)
for layer in model_tr.layers [:-3]:
    layer.trainable=True #將鎖定的層打開，參與訓練

In [31]:
model_tr.summary(show_trainable=True)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ flatten (Flatten)           │ (None, 784)           │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 200)           │    157,000 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 150)           │     30,150 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 100)           │     15,100 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_10 (Dense)            │ (None, 80)            │      8,080 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_11 (Dense)            │ (None, 50)            │      4,050 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_12 (Dense)            │ (None, 1)             │         51 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 214,433 (837.63 KB)

 Trainable params: 214,431 (837.62 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [32]:
model_tr.compile(loss='binary_crossentropy',
                 optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
                 metrics= ['accuracy'])

In [33]:
train = model_tr.fit(x_train_B, y_train_B, epochs=20,
                      validation_data=(x_valid_B, y_valid_B))

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.9691 - loss: 0.3991 - val_accuracy: 0.9865 - val_loss: 0.3900
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.3819 - val_accuracy: 0.9896 - val_loss: 0.3752
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.3667 - val_accuracy: 0.9909 - val_loss: 0.3618
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.3531 - val_accuracy: 0.9916 - val_loss: 0.3496
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.3407 - val_accuracy: 0.9936 - val_loss: 0.3383
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.3293 - val_accuracy: 0.9943 - val_loss: 0.3279
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.3188 - val_accuracy: 0.9946 - val_loss: 0.3181
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.3089 - val_accuracy: 0.9949 - val_loss: 0.3088


In [34]:
model_tr.evaluate(x_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.9981 - loss: 0.2207


[0.22050534188747406, 0.9990000128746033]

In [35]:
model_B.evaluate(x_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - accuracy: 0.5218 - loss: 0.5816


[0.5923401117324829, 0.5009999871253967]